In [17]:
%load_ext autoreload
%autoreload 2

from dynamic_system_functions import *
import casadi as ca
cax = ca.MX
M = cax.sym('M')
B = cax.sym('B')
Kp = cax.sym('Kp')
Kd = cax.sym('Kd')

R = sys([Kd,Kp,0],[M,B,Kp])

Delay = sys([1],[1])
#Delay = pade(2e-3, 2)

Kl = cax.sym('Kl')
Cff = sys([Kl, 0], [1])

B_real  = cax.sym('B_real')
a_vel   = cax.sym('a_vel')
P_hat   = cax.sym('P_hat')
a_acc   = cax.sym('a_acc')
C_s_inv = sys([B_real*a_vel, 0],[1, a_vel, 0])
P_hat_Oa_s_inv = sys([-P_hat*a_acc*a_vel, 0, 0],[1, a_acc+a_vel, a_acc*a_vel, 0])

Ma = cax.sym('Ma')
Ba = cax.sym('Ba')
A = sys([1], [Ma, Ba, 0])

p = [M, B, Kp, Kd] # fixed params
x = [Kl, Ma, Ba, B_real, a_vel, P_hat, a_acc]   # dec variables
vars = [*p, *x]    # list of all the variables, useful for making casadi functions
x_names = [str(v) for v in x]

##### Building some SYSTEMS ######
# OL from F to v
G = R*Delay*(1+Cff)*A

# Feedback from payload comp and control
D = R*Delay*C_s_inv + G*P_hat_Oa_s_inv

# Fd to v with feedback
G_re, _ = (G/(1+D)).get_re_im(vars)

# Environment contact dynamics
K_env = cax.sym('K_env')
M_env = cax.sym('M_env')
E = sys([M_env, 0, K_env], [1, 0])

G_cl = G/(1+D+E*G)
G_cl_den_fn = ca.Function('G_cl_den_fn', [K_env, M_env, *vars], G_cl.den)


# Noise TFs
noise_area_of_concern = sys([100, 0], [1, 100])
G_nf = Cff*A*noise_area_of_concern
G_nx = (C_s_inv + Cff*A*(P_hat_Oa_s_inv))*noise_area_of_concern

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Cancelling pole/zero at 0


In [25]:
# Formulate optimization objective:
# maximize admittance 
# s.t.:
p0  = [5, 120, 400, 100] # numerical value of params
x0  = [0.01, 12,  500,  0.01, 20, 18, 10]  # initial guess at dec vars
vars0 = [*p0, *x0]
ubx = [0.06,  30, 3000,  1.0,  60,  40,  60]  # upper bound
lbx = [ 0.0,   5,  100,  0.0,  0.0,  0.0, 0.0]# lower bound
noise_bound_f = 5e-3 # 1e-3 is more realistic, but let's see
noise_bound_x = 1e-1 # 1e-3 is more realistic, but let's see
passivity_bound = -1e-3
M_en = 12.0        # Env mass in Kg

h2_solver = 'scipy'

# OBJECTIVE: J will be minimized, to maximize admittance F->v
J = -G.h2(sol = h2_solver) 

# Scale the objective
J_fn = ca.Function('J_fn', [*vars], [J])
J0 = J_fn(*p0, *x0)
J *= 1/ca.fabs(J0)
J_grad_fn = ca.Function('J_grad_fn', [*vars], [ca.gradient(J, ca.horzcat(*x))])
print('Objective: {}'.format(J0))
print('Gradient:  {}'.format(J_grad_fn(*p0, *x0)))

# CONSTRAINTS
g = []
ubg = []
lbg = []

# OL Stability:
rt = instab(G.den)
for r_i in range(1, len(rt)):
    g.append(rt[r_i]*rt[0])
    ubg.append(np.Inf)
    lbg.append(1e-8) 
    
# Contact Stability:
for K_en in [5e3, 5e4, 5e5, 5e6, 5e7]:
    #Stab
    G_cl_den = G_cl_den_fn(K_en, M_en, *vars)
    rt = instab(G_cl_den)
    for r_i in range(1, len(rt)):
        g.append(rt[r_i]*rt[0])
        ubg.append(np.Inf)
        lbg.append(1e-8)    

#Noise
g.append(G_nf.h2(sol = h2_solver))
ubg.append(noise_bound_f)
lbg.append(-np.Inf)
    
g.append(G_nx.h2(sol = h2_solver))
ubg.append(noise_bound_x)
lbg.append(-np.Inf)

for om in np.logspace(0.1, 4, num = 50):
    imag_pt = G_re(om, *p0, *x)
    g.append(imag_pt)
    lbg.append(passivity_bound)
    ubg.append(np.Inf)
    
    
opts = { 'ipopt.print_level' : 0,
         'ipopt.hessian_approximation': 'exact',#'limited-memory',
         'ipopt.mu_strategy': 'monotone',         # Determines which barrier parameter update strategy is to be used. Default "monotone", can choose 'adaptive' instead
         'ipopt.mu_init' : 1e-3,                  # This option determines the initial value for the barrier parameter (mu). It is only relevant in the monotone, Fiacco-McCormick version of the algorithm.
         'ipopt.expect_infeasible_problem' : 'yes',
}
        
# J is objective, w decision vars, g constraints (subject to ubg and lbg)
prob = dict(f = J, x = ca.vertcat(*x), g = ca.vertcat(*g), p = ca.vertcat(*p))
solver = ca.nlpsol('solver', 'ipopt', prob, opts)
args = dict(x0 = x0, lbx = lbx, ubx = ubx, ubg = ubg, lbg = lbg, p = p0)
sol = solver(**args)
x_opt = sol['x'].full().tolist()
for val, nam in zip(x_opt, x_names):
    print('{:7s} : {:.3f}'.format(nam, val[0]))

G.nyquist(vars, [*p0, *sol['x'].full()])

Objective: -0.00541103
Gradient:  [[-7.69231, 0.0167186, 0.00159875, 0, 0, 0, 0]]


RuntimeError: .../casadi/core/function_internal.cpp:145: Error calling IpoptInterface::init for 'solver':
Error in Function::factory for 'nlp' [MXFunction] at .../casadi/core/function.cpp:1634:
Failed to create nlp_hess_l:[x, p, lam:f, lam:g]->[hess:gamma:x:x] with {"gamma": [f, g]}:
.../casadi/core/factory.hpp:387: Hessian generation failed:
Error in MX::hessian at .../casadi/core/mx.cpp:1679:
Error in MX::jacobian at .../casadi/core/mx.cpp:1663:
Error in XFunction::jac for 'helper_jacobian_MX' [MXFunction] at .../casadi/core/x_function.hpp:719:
Error in MXFunction::ad_forward at .../casadi/core/mx_function.cpp:831:
Error in MX::ad_forward for node of type N6casadi4CallE at .../casadi/core/mx.cpp:2035:
Error in Call::ad_forward for 'adj1_lyap_solver' [CallbackInternal] at .../casadi/core/casadi_call.cpp:123:
.../casadi/core/function_internal.cpp:2639: Assertion "has_derivative()" failed:
Derivatives cannot be calculated for adj1_lyap_solver

In [ ]:
# Find extrema of imaginary component over frequency
imag_der = der(imag_coeff)
print(imag_coeff)
poly_coeffs = ca.SX.sym('poly',3,1)
poly_coeffs[0] = imag_der[0]
poly_coeffs[1] = imag_der[2]
poly_coeffs[2] = imag_der[4]
roots = ca.sqrt(-ca.poly_roots(poly_coeffs))
print(roots[0])

for j in range(roots.shape[0]):
    tot = 0.0
    num_coeffs = len(imag_coeff)
    for i in range(num_coeffs):
        tot += ca.constpow(roots[j],num_coeffs-i-1)*imag_coeff[i]
    print(tot)
crit_points_imag = []
crit_points_real = []
crit_points_freq = []
for i in range(roots.shape[0]):
    crit_points_freq.append(ca.Function('crit_freq', [M, B, Kp, Kd, Ma, Ba, Kl], [roots[i]]))
    crit_points_imag.append(ca.Function('crit_imag', [M, B, Kp, Kd, Ma, Ba, Kl], [imag_fn(roots[i], M, B, Kp, Kd, Ma, Ba, Kl)]))
    crit_points_real.append(ca.Function('crit_real', [M, B, Kp, Kd, Ma, Ba, Kl], [real_fn(roots[i], M, B, Kp, Kd, Ma, Ba, Kl)]))

# Verifying the real/imaginary components are realistic
# Can compare with addmittance_check_casadi.m
import numpy as np
import matplotlib.pyplot as plt
Mt = 5
Bt = 10
Kpt = 1
Kdt = 2
Mat = 3
Bat = 5
Klt = 0.1

plt.figure()
plt.clf()
for om in [0.001, 10, 500]:
    print("freq {} real {} im {}".format(om, real_fn(om, Mt, Bt, Kpt, Kdt, Mat, Bat, Klt), imag_fn(om, Mt, Bt, Kpt, Kdt, Mat, Bat, Klt)))

for om in np.logspace(-10,10, num = 1000):
    plt.plot(real_fn(om, Mt, Bt, Kpt, Kdt, Mat, Bat, Klt), imag_fn(om, Mt, Bt, Kpt, Kdt, Mat, Bat, Klt),'ko')
    
for cp_r, cp_i, cp_f in zip(crit_points_real, crit_points_imag, crit_points_freq):
    print('crit point re: {} im: {} fr: {}'.format(\
                        cp_r(Mt, Bt, Kpt, Kdt, Mat, Bat, Klt), 
                        cp_i(Mt, Bt, Kpt, Kdt, Mat, Bat, Klt),
                        cp_f(Mt, Bt, Kpt, Kdt, Mat, Bat, Klt)))
    plt.plot(cp_r(Mt, Bt, Kpt, Kdt, Mat, Bat, Klt), cp_i(Mt, Bt, Kpt, Kdt, Mat, Bat, Klt),'dr')
plt.show()

In [16]:
# TEST FUNCTIONS
# Test instap
#RT_col = instab([1, 2, 3, 4, 5])
#print(RT_col)

# Test lyap
A = np.array([[-0.1, 0, 1],[0, 1, 0], [0, 0, 1]])
Q = np.array([[1, 0, 0],[0, 1, 0], [0, 3, 1]])
X, _ = lyap(A,Q)
print('LYAP')
print(X)

# Test H2
A = np.array([[-0.5, 1],[-0.2, -0.1]])
B = np.array([[0],[.3]])
C = np.array([[3, 0]])
h2_res, _ = h2(A, B, C)
print('H2')
print(h2_res)

# Test tf2ss
#num = [1, 3]
#den = [1, 5, 10, 20, 20]
#A, B, C = tf2ss(num, den)
#print(h2(A, B, C))

# Test symbolic diff
b = ca.MX.sym('b')
k = ca.MX.sym('k')
A = ca.MX.zeros(2,2)
A[0,1] = ca.MX(1.0)
A[1,0] = -b
A[1,1] = -k
#Q = ca.MX.eye(2)
norm, sol = h2(A, B, C, 'lapackqr')
norm_fn = ca.Function('norm', [b, k], [norm])
b_grad_fn = ca.Function('b_grad',[b, k], [ca.gradient(norm,b)])
k_grad_fn = ca.Function('k_grad',[b, k], [ca.gradient(norm,k)])

b0 = 2
k0 = 1.0
h = 0.01
h20 = norm_fn(b0, k0)
bgrad = b_grad_fn(b0, k0)
kgrad = k_grad_fn(b0, k0)
h21 = norm_fn(b0, k0+h*bgrad+h*kgrad)


print('H2 init: {}'.format(h20))
print('Grads:   {}, {}'.format(bgrad, kgrad))
print('H2 fin:  {}'.format(h21))
print('err:     {}'.format(h20+h*bgrad+h*kgrad-h21))

LYAP

[[10.5556, 1.66667, 0.555556], 
 [0, -0.5, 0], 
 [0.555556, -1.5, -0.5]]
H2
1.64317
H2 init: 0.45
Grads:   -0.1125, -0.225
H2 fin:  0.450761
err:     -0.0041363


In [13]:
# Testing the lyap_solver comparing with the vanilla Casadi Solve
from dynamic_system_functions import sys
dummy_sys = sys([],[])
ly = lyap_solver('lyapio',2)
A = np.array([[0.1, 1],[1, 3]])
Q = np.eye(2)
B = np.array([[1.3, 0.1],[0.1, 5]])
X, _ = lyap(A, B, 'scipy')
print('Scipy lyap:')
print(X)
X, _ = lyap(A, B, 'lapackqr')
print('Casadi lyap:')
print(X)


A = np.array([[-0.5, 1],[-0.2, -0.1]])
#B = np.array([[0],[.3]])
#C = np.array([[3, 0]])
n, _ = h2(A, B, C, 'scipy')
print('Scipy h2')
print(n)
n, _ = h2(A, B, C, 'lapackqr')
print('Casadi h2')
print(n)


As = ca.MX.sym('As',2,2)
Qs = ca.MX.sym('Qs',2,2)
A2s = ca.MX.sym('As',2,2)
Q2s = ca.MX.sym('Qs',2,2)
Ps = ca.MX.sym('Ps',2,2)
fseeds = [ca.MX.sym("f",2,2) for i in range(ly.n_in())]
aseeds = [ca.MX.sym("a",2,2)  for i in range(ly.n_out())]
#res = ly.call([As, Qs], True)
#res = ly(As, Qs)
res, sol = lyap(As, Qs, 'scipy')
res_ca, _ = lyap(As, Qs, 'lapackqr')

[rev] = ca.reverse([res], [As, Qs, Ps, A2s, Q2s], [aseeds], {})
[rev_ca] = ca.reverse([res_ca], [As, Qs, Ps, A2s, Q2s], [aseeds], {})

[fwd] = ca.forward([res], [As, Qs], [fseeds], {})
[fwd_ca] = ca.forward([res_ca], [As, Qs], [fseeds], {})

fwd_fn = ca.Function('fwfn', [As, Qs, *fseeds], fwd)
fwd_ca_fn = ca.Function('fwfn', [As, Qs, *fseeds], fwd_ca)
rev_fn = ca.Function('rvfn', [As, Qs, *aseeds], rev)
rev_ca_fn = ca.Function('rvfn', [As, Qs, *aseeds], rev_ca)
print('Validating lyap fwd')
print('Scipy')
print(fwd_fn(A, Q, A, B))
print('Casadi')
print(fwd_ca_fn(A, Q, A, B))
print('Validating lyap rev')
print('Scipy')
print(rev_fn(A, Q, B))
print('Casadi')
print(rev_ca_fn(A, Q, B))

Scipy lyap:

[[3.5, -1], 
 [-1, -0.5]]
Casadi lyap:

[[3.5, -1], 
 [-1, -0.5]]
Scipy h2
27.7009
Casadi h2
27.7009
Validating lyap fwd
Scipy

[[13.66, 6.68], 
 [6.68, 6.64]]
Casadi

[[13.66, 6.68], 
 [6.68, 6.64]]
Validating lyap rev
Scipy

[[-0.5, 1], 
 [-0.2, -0.1]]

[[1, 0], 
 [0, 1]]

[[4.2, 1.6], 
 [1.6, 1.8]]
(DM(
[[11.112, 1.376], 
 [31.52, 42.96]]), DM(
[[1.78, -1.2], 
 [-1.2, 13]]), DM(
[[00, 00], 
 [00, 00]]), DM(
[[00, 00], 
 [00, 00]]), DM(
[[00, 00], 
 [00, 00]]))
Casadi
(DM(
[[11.112, 1.376], 
 [31.52, 42.96]]), DM(
[[1.78, -1.2], 
 [-1.2, 13]]), DM(
[[00, 00], 
 [00, 00]]), DM(
[[00, 00], 
 [00, 00]]), DM(
[[00, 00], 
 [00, 00]]))
